In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_train = pd.read_csv('../data/train_cleaned.csv', index_col=0)
budget_test = pd.read_csv('../data/test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
budget_train.isnull().sum()

Function                      0
Use                           0
Sharing                       0
Reporting                     0
Student_Type                  0
Position_Type                 0
Object_Type                   0
Pre_K                         0
Operating_Status              0
Object_Description          423
Text_2                       17
SubFund_Description           0
Job_Title_Description         0
Text_3                    70814
Text_4                       18
Sub_Object_Description        0
Location_Description          0
FTE                           0
Function_Description          5
Facility_or_Department        0
Position_Extra              611
Total                         0
Program_Description           2
Fund_Description              0
Text_1                      422
combined_text                 0
dtype: int64

In [4]:
budget_test.isnull().sum()

ID                            0
Object_Description         1752
Program_Description        5253
SubFund_Description       33953
Job_Title_Description     17747
Facility_or_Department    47225
Sub_Object_Description    16452
Location_Description      12748
FTE                           0
Function_Description       3198
Position_Extra            36279
Text_4                    47252
Total                         0
Text_2                    45426
Text_3                    44281
Fund_Description          10478
Text_1                    34717
dtype: int64

In [5]:
for col in budget_test[word_predictors]:
    budget_test[col] = budget_test[word_predictors].loc[:,[col]].fillna(' ')

In [6]:
for col in budget_train[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [7]:
label = LabelEncoder()

In [8]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    

In [9]:
class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return np.transpose(np.matrix(data))

In [10]:
targets

['Function',
 'Use',
 'Sharing',
 'Reporting',
 'Student_Type',
 'Position_Type',
 'Object_Type',
 'Pre_K',
 'Operating_Status']

In [11]:
# 400,000+ examples, 14 features
X = budget_train[predictors] 
y = label.fit_transform(budget_train['Function'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state=42)
                                                

In [12]:
# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
#     ('FTE', Pipeline([
#                 ('selector', ItemSelector(key='FTE')),
#                 ('scaler', StandardScaler()),
#                 ('caster', ArrayCaster()),
#             ])),
#     ('Total', Pipeline([
#                 ('selector', ItemSelector(key='Total')),
#                 ('scaler', StandardScaler()),
#                 ('caster', ArrayCaster()),
#             ])),
]

### Don't include numerical columns in final feature union since it decreased scores and increased log loss- in general, in the original data, numerical features had a lot of missing values and some incorrect values for data that were there

In [13]:
# create pipeline
model_pipeline = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression()),
])

In [14]:
X_test = budget_test

In [15]:
function_model = model_pipeline.fit(X_train, y_train)

In [16]:
function_model.score(X_train, y_train)

0.95933433472508045

In [17]:
val_proba = function_model.predict_proba(X_val)

In [18]:
predict_proba = function_model.predict_proba(X_test)

In [12]:
label.classes_

array(['Function__Aides Compensation',
       'Function__Career & Academic Counseling',
       'Function__Communications', 'Function__Curriculum Development',
       'Function__Data Processing & Information Services',
       'Function__Development & Fundraising', 'Function__Enrichment',
       'Function__Extended Time & Tutoring',
       'Function__Facilities & Maintenance',
       'Function__Facilities Planning',
       'Function__Finance, Budget, Purchasing & Distribution',
       'Function__Food Services', 'Function__Governance',
       'Function__Human Resources',
       'Function__Instructional Materials & Supplies',
       'Function__Insurance', 'Function__Legal',
       'Function__Library & Media', 'Function__NO_LABEL',
       'Function__Other Compensation', 'Function__Other Non-Compensation',
       'Function__Parent & Community Relations',
       'Function__Physical Health & Services',
       'Function__Professional Development', 'Function__Recruitment',
       'Function__Rese

In [21]:
log_loss(y_val, val_proba)

0.16233774848612828

In [22]:
from sklearn.externals import joblib

In [25]:
joblib.dump(function_model, 'function_model.pkl')

['function_model.pkl']

In [26]:
def score_eval(model, X_val, y_val):
    preds = model.predict(X_val)
    proba = model.predict_proba(X_val)
    return {
        'log_loss': log_loss(y_val, proba),
        'accuracy': accuracy_score(y_val, preds),
        'confmat': confusion_matrix(y_val, preds),
        'clf_rep': classification_report(y_val, preds)
    }

In [28]:
results = score_eval(function_model, X_val, y_val)

print('-'*60)
for key, value in results.items():
    print(key)
    print(value)
    print('-'*60)

------------------------------------------------------------
log_loss
0.162337748486
------------------------------------------------------------
accuracy
0.955106425502
------------------------------------------------------------
confmat
[[ 3925     0     0 ...,     4     0     0]
 [    2   493     0 ...,    24     1     0]
 [    0     0    45 ...,     1     0     0]
 ..., 
 [   15     0     0 ..., 16947     0     0]
 [    0     0     0 ...,    15    15     0]
 [    0     0     1 ...,     0     0   342]]
------------------------------------------------------------
clf_rep
             precision    recall  f1-score   support

          0       0.97      0.99      0.98      3956
          1       0.86      0.87      0.86       566
          2       0.83      0.80      0.82        56
          3       0.91      0.86      0.88      1514
          4       0.94      0.88      0.91       678
          5       0.80      0.44      0.57        18
          6       0.95      0.85      0.90      

In [13]:
classes = label.classes_

In [14]:
classes

array(['Function__Aides Compensation',
       'Function__Career & Academic Counseling',
       'Function__Communications', 'Function__Curriculum Development',
       'Function__Data Processing & Information Services',
       'Function__Development & Fundraising', 'Function__Enrichment',
       'Function__Extended Time & Tutoring',
       'Function__Facilities & Maintenance',
       'Function__Facilities Planning',
       'Function__Finance, Budget, Purchasing & Distribution',
       'Function__Food Services', 'Function__Governance',
       'Function__Human Resources',
       'Function__Instructional Materials & Supplies',
       'Function__Insurance', 'Function__Legal',
       'Function__Library & Media', 'Function__NO_LABEL',
       'Function__Other Compensation', 'Function__Other Non-Compensation',
       'Function__Parent & Community Relations',
       'Function__Physical Health & Services',
       'Function__Professional Development', 'Function__Recruitment',
       'Function__Rese

In [32]:
function_probability = pd.DataFrame(predict_proba, columns=classes)

In [36]:
function_probability.head()

,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,Function__Facilities Planning,...,Function__School Supervision,Function__Security & Safety,Function__Social & Emotional,Function__Special Population Program Management & Support,Function__Student Assignment,Function__Student Transportation,Function__Substitute Compensation,Function__Teacher Compensation,Function__Untracked Budget Set-Aside,Function__Utilities
0,0.003283,0.000519,0.000011,0.000254,0.000604,0.000052,0.003436,0.000141,0.006110,0.000064,...,0.000135,0.000216,0.000291,0.001048,0.000217,0.275061,0.000459,0.006293,0.001564,0.006130
1,0.000119,0.001974,0.000316,0.002526,0.009157,0.000285,0.372242,0.000882,0.098478,0.000367,...,0.000081,0.008704,0.011119,0.007580,0.000534,0.000131,0.000086,0.002834,0.001556,0.002551
2,0.057924,0.002755,0.000062,0.002482,0.002392,0.000102,0.001242,0.000675,0.013308,0.000255,...,0.000094,0.002743,0.004124,0.000926,0.000438,0.001982,0.002469,0.008120,0.002062,0.003788
3,0.058561,0.002759,0.000063,0.002485,0.002395,0.000102,0.001243,0.000674,0.013332,0.000255,...,0.000094,0.002747,0.004130,0.000928,0.000438,0.001984,0.002463,0.008024,0.002066,0.003794
4,0.000459,0.015215,0.000142,0.000264,0.003094,0.000145,0.023989,0.000053,0.018626,0.000131,...,0.000206,0.000959,0.000245,0.000519,0.000271,0.000871,0.000017,0.000452,0.000268,0.000237


In [37]:
function_probability.to_csv('function_probability.csv')